In [ ]:
# this cell produces a list with countries that have data available
# some additional information is printed to the output
from unfccc_ghg_data.unfccc_crf_reader.unfccc_crf_reader_core import get_country_folders, get_latest_version_for_country
from unfccc_ghg_data.helper import all_countries

# definitions:
submission_round = 1

countries_to_read = []
for country in all_countries:
    try:
        folders = get_country_folders([country], submission_year=submission_round, submission_type="CTF")
        existing = [folder.exists() for folder in folders]
        if any(existing):
            try:
                version = get_latest_version_for_country(country, submission_round=submission_round, submission_type="CTF")
                print(f"{country}: {version}")
                countries_to_read.append(country)
            except Exception as ex:
                print(f"{country}: BTR but no CTF found: {ex}")
        else:
            print(f"{country}: no BTR found")
    except Exception as ex:
        print(f"{country}: No folder found: {ex}")

len(countries_to_read)

In [ ]:
from unfccc_ghg_data.unfccc_crf_reader.unfccc_ctf_reader import read_ctf_for_country
from unfccc_ghg_data.unfccc_crf_reader.util import NoCRFFilesError

read_countries = {}
ds_all = None
#countries_to_read = ['DEU']
for country in countries_to_read:
    try:
        ds_country = read_ctf_for_country(
            country,
            submission_round=submission_round,
            re_read=True,  # currently not doing anything as CTF data not stored per country
        )
        if ds_country is None:
            read_countries[country] = "skipped"
        else:
            read_countries[country] = "read"
            if ds_all is None:
                ds_all = ds_country
            else:
                ds_all = ds_all.merge(ds_country)
    except NoCRFFilesError:
        print(f"No CTF data for country {country}, {submission_round}")
        read_countries[country] = "no data"
    except Exception as ex:
        print(
            f"CTF data for country {country}, "
            f"{submission_round} could not be read"
        )
        print(f"The following error occurred: {ex}")
        read_countries[country] = "failed"

# print overview
successful_countries = [
    country for country in read_countries if read_countries[country] == "read"
]
skipped_countries = [
    country for country in read_countries if read_countries[country] == "skipped"
]
failed_countries = [
    country for country in read_countries if read_countries[country] == "failed"
]
no_data_countries = [
    country for country in read_countries if read_countries[country] == "no data"
]

print(f"Read data for countries {successful_countries}")
print(f"Skipped countries {skipped_countries}")
print(f"No data for countries {no_data_countries}")
print(f"!!!!! Reading failed for {failed_countries}. Check why")


In [ ]:
ds_all

In [ ]:
from unfccc_ghg_data.unfccc_crf_reader.unfccc_ctf_reader import read_to_test_specs_CTF
#data_test = read_year_to_test_specs_CTF(submission_year=1, country_code='USA')
data_test = read_to_test_specs_CTF(submission_round=1)

In [ ]:
# test a specific table without exception handling so we get more error detail
from unfccc_ghg_data.unfccc_crf_reader.unfccc_ctf_reader import read_ctf_for_country, read_ctf_table
from unfccc_ghg_data.unfccc_crf_reader.unfccc_crf_reader_core import get_latest_version_for_country

country_test = 'USA'
table_test = 'Table7'
submission_round = 1

version = get_latest_version_for_country(
    country_test, submission_round=submission_round, submission_type='CTF'
)


(
    ds_table,
    new_unknown_categories,
    new_last_row_info,
    not_present,
) = read_ctf_table(
    country_test,
    table_test,
    submission_round,
    version=version,
    debug=True,
)



In [ ]:

ds_country = read_ctf_for_country(
    country_test, 
    submission_round=submission_round,
    version=version,
)

In [ ]:
ds_table